# Dopasowywanie wyłuskanych za pomocą NLP sąsiadujących słów kluczowych do właściwego modelu danych:

In [ ]:
# Przykładowy rekord
model = {'type':"Lokata",
         'name':"Lokata mobilna plus",
         'duration(months)': 12,
         'percentage': 4,
         'customer_type': "Indywidualny",
         'offer_type': "Dla nowych środków",
         'max_deposit_amount':200000,
         'bank': "mBank"}

Za pomocą wyuczonego modelu NLP z tekstu dokumentu wyfiltrowanego ze strony banku wyłuskane zostają słową kluczowe. Następnie, taki zestaw jest klasyfikowany przez algorytmem Naive Bayes i dopasowywany do odpowiednich etykiet.

Model Naive Bayes uczony był za pomocą specjalnie wygenerowanych losowo danych, uwzględniających wszystkie banki które zostały wskazane przez organizatorów wydarzenia.

Fragment pliku .csv z wygenerowanymi danymi:

| type   | name         | duration(months) | percentage | customer_type | offer_type         | max_deposit_amount | bank         |
|--------|--------------|-------------------|------------|----------------|--------------------|--------------------|--------------|
| Lokata | Super Lokata | 11                | 4.55       | Korporacyjny   | Dla nowych srodkow | 44848              | mBank        |



Algorytm Naive Bayes osiągnął świetny wynik po przetestowaniu go za pomocą zbioru testowego (100% właściwego dopasowania). Dane w zbiorze testowym pochodziły jednak ze sztucznie wygenerowanego zestawu danych. 

### Generowanie danych wykorzyzstanych do uczenia:

In [ ]:
import random
import pandas as pd 

def generate_random_financial_product():
    # 1. Typ
    product_type = random.choice(['Konto oszczędnościowe', 'Lokata'])
    
    # 2. Nazwa
    product_name = random.choice(['Super Lokata', 'Mega Konto Oszczędnościowe', 'Elastyczna Inwestycja', 'Premium Plus'])

    # 3. Długość (w miesiącach)
    duration_months = random.randint(1, 60)  # Losowa liczba miesięcy od 1 do 60

    # 4. Wysokość oprocentowania (zazwyczaj w okolicach 4-5%)
    interest_rate = round(random.uniform(4, 5), 2)  # Losowa wartość z zakresu 4-5 z dwoma miejscami po przecinku

    # 5. Typ klienta
    customer_type = random.choice(['Indywidualny', 'Korporacyjny', 'Samorzad'])

    # 6. Rodzaj oferty
    offer_type = random.choice(['Dla nowych', 'Dla wszystkich', 'Dla nowych srodkow'])

    # 7. Maksymalna kwota depozytu
    max_deposit_amount = random.randint(1000, 100000)  # Losowa kwota od 1000 do 100 000

    # 8. Bank
    banks = ['ING Bank Slaski', 'Alior Bank', 'mBank', 'Citibank', 'PEKAO S.A', 'PKO B.P', 'Bank Pocztowy S.A.',
             'Nest Bank', 'Plus Bank', 'Santander Bank Polska', 'Santander Consumer Bank', 'Toyota Bank Polska',
             'VeloBank S.A.', 'Bank Nowy', 'Bank Millenium', 'Bank Ochrony Srodowiska', 'Deutsche Bank Polska']
    bank = random.choice(banks)

    return {
        'type': product_type,
        'name': product_name,
        'duration [months]': duration_months,
        'percentage': interest_rate,
        'customer_type': customer_type,
        'offer_type': offer_type,
        'max_deposit_amount': max_deposit_amount,
        'bank': bank
    }

def save_to_csv(file_path, data):
    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False, encoding='utf-8')


num_records = 1024
generated_data = [generate_random_financial_product() for _ in range(num_records)]

csv_file_path = 'data_to_learn_model.csv'
save_to_csv(csv_file_path, generated_data)

#### Klasyfikacja danych za pomocą modelu Naive Bayes:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score

# Lista etykiet
labels = ["type", "name", "duration(months)", "percentage", "customer_type", "offer_type", "max_deposit_amount", "bank"]

# Słownik mapujący etykiety na liczby całkowite
labels_mapping = {'type': 0, 'name': 1, 'duration(months)': 2, 'percentage': 3, 'customer_type': 4, 'offer_type': 5, 'max_deposit_amount': 6, 'bank': 7}

# Wczytanie pliku CSV do ramki danych
csv_file_path = 'D:/Projekty/Modules/scrapper/data/data_to_learn_model.csv'
df = pd.read_csv(csv_file_path)

# Utworzono listę, do której będziemy dodawać pojedyncze stringi
all_texts = df['name'].astype(str).tolist()

# Przekształcenie etykiet na liczby całkowite
y = df[labels].apply(lambda row: [labels_mapping.get(key, 'Unknown') for key in row], axis=1)

# Wykorzystanie MultiLabelBinarizer do przekształcenia sekwencji etykiet na macierz binarną
mlb = MultiLabelBinarizer()
y_train_binary = mlb.fit_transform(y)

# Podzielono dane na zbiór treningowy i testowy
X_train, X_test, y_train_binary, y_test = train_test_split(all_texts, y_train_binary, test_size=0.2, random_state=42)

# Przygotowane wektory cech za pomocą CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Trenowanie modelu klasyfikacyjnego:
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train_binary)

# Ocena skuteczności modelu na zbiorze testowym:
y_pred = clf.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")